In [22]:
### Illustration du biais de regularisation
### 26 septembre 2019
### Jeremy LHour

library("MASS")
library("glmnet")

# fonctions utilisateur
source("functions/DataSim.R") 

PostLasso <- function(X,y,nopen){
    #'@param nopen indices des variables a ne pas penaliser
    phi = rep(1,ncol(X)); phi[nopen] = 0
    cv.lasso = cv.glmnet(X,y, family="gaussian",alpha=1,penalty.factor=phi,nfolds=5)
    b.lasso = coef(cv.lasso); b.lasso = b.lasso[-1] # on enleve la constante
    s.hat = which(b.lasso != 0) # indices des variables actives parmi les X
    print(s.hat)
    postlasso = lm(y ~ X[,s.hat])
    b.pl = coef(postlasso)

    return(b.pl)
}

PostStudent <- function(X,y,noppen,alpha=.05){
    #'@param nopen indices des variables a ne pas penaliser
    #'@param alpha niveau de significativite pour selection
    fullreg = lm(y ~ X)
    s.hat = summary(fullreg)$coef[,"Pr(>|t|)"] < alpha
    s.hat = s.hat[-1]
    s.hat = union(s.hat,nopen)
    shortreg = lm(y ~ X[,s.hat])
    b.ps = coef(shortreg)

    return(b.ps)
}

In [21]:
### Simulations
R = 1000 # nb simulations
n = 2000 # sample size
p = 10 # nb variables

set.seed(12071990)
data = DataSim(n, p, Ry= .2, Intercept=F)
y = data$y; X = data$X


Call:
lm(formula = y ~ X)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.5287 -0.6794  0.0006  0.7013  3.9221 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.1850646  0.0920817   2.010   0.0446 *  
X1          -0.5627363  0.0293335 -19.184   <2e-16 ***
X2           0.0478716  0.0593548   0.807   0.4200    
X3          -0.0531110  0.0257007  -2.067   0.0389 *  
X4          -0.0124577  0.0624004  -0.200   0.8418    
X5          -0.0127931  0.0531626  -0.241   0.8099    
X6           0.0797145  0.0610825   1.305   0.1920    
X7           0.0368171  0.0464784   0.792   0.4284    
X8           0.0750002  0.0653359   1.148   0.2511    
X9           0.0001293  0.0299675   0.004   0.9966    
X10          0.0337080  0.0578976   0.582   0.5605    
X11         -0.0156228  0.0233607  -0.669   0.5037    
X12          0.0716811  0.0619533   1.157   0.2474    
X13         -0.0511413  0.0405358  -1.262   0.2072    
X14          0.0506259  0.0600868   0.8

(Intercept) X[, s.hat]1 X[, s.hat]2 
 0.31123998 -0.57141236 -0.02395663 
